Script to generate files needed for manual annotation. 

0) mount drive

1) Run block #1.

2) Change the sample name in block #2 and run. Will generate annotation files for all image blocks for that sample. Output is in /Annotation/

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1) setup
#load packages
import pandas as pd
import numpy as np
from skimage import io,util, measure
import shutil,re,datetime
import os
import sys
import dill
!pip install ipdb
import ipdb

# load function for fatetrack objects
def load_object(filename):
   with open(filename, 'rb') as red:  # Overwrites any existing file.
       tmp = dill.load(red)
   return(tmp)

# function to generate all needed files for annotation
def generateAnnotationFiles(sampleBase, block):
    # make the file
    sample = sampleBase + "_9trailing_block" + str(block) + "of9"
    fto = load_object('/content/drive/MyDrive/FateTrack_Main/test/0411/' + sample + '/' + sample + '.pkl')
    mask = fto.HCR_Rd1_DAPI_mask
    measures = pd.DataFrame(measure.regionprops_table(mask,properties=('label','centroid')))
    measDF = measures[['label','centroid-0','centroid-1']].astype(int)
    measDF.columns = ['Rd1_orig_label', 'yCoord', 'xCoord']
    measDF['annotation'] = 'none'
    #hcrs = fto.HCR_measurements
    hcrs = pd.read_csv('/content/drive/MyDrive/FateTrack_Main/test/0411/' + sample + '/hcr/' + sample + '_HCRmeasures_dil9.csv')
    hcrs_nuc_meas = hcrs.filter(like = "mean").filter(like = "nuc")
    hcrs_nuc_meas = hcrs_nuc_meas * 10000
    hcrs = pd.concat([pd.DataFrame(hcrs['Rd1_orig_label']), hcrs_nuc_meas, hcrs.filter(like = "mean").filter(like = "cyto")], axis = 1).astype(int)
    measDF = pd.merge(measDF, hcrs, on = "Rd1_orig_label")
    measDF.rename(columns = {'Rd1_orig_label' : 'pointID'}, inplace = True)
    # make a directory to save the files to
    path = '/content/drive/MyDrive/FateTrack_Main/test/0411/' + sampleBase + '/' + sample
    if not os.path.isdir(path):
      os.makedirs(path)
    # save the annotation file and the aligned tifs to the directory
    measDF.to_csv(path + "/" + sample + "_annotations.csv", index = False)
    images1 = fto.HCR_Rd1_images
    images2 = fto.HCR_Rd2_images
    io.imsave(path + "/mask.tif", util.img_as_uint(mask>0))
    # io.imsave(path + "/cyto_mask.tif", util.img_as_uint(fto.HCR_Rd1_cyto_mask>0))
    io.imsave(path + "/1CY5.tif", images1[np.where(fto.HCR_channelList=='CY5')[0][0]])
    io.imsave(path + "/1CY3.tif", images1[np.where(fto.HCR_channelList=='CY3')[0][0]])
    io.imsave(path + "/1YFP.tif", images1[np.where(fto.HCR_channelList=='YFP')[0][0]])
    io.imsave(path + "/1DAPI.tif", images1[np.where(fto.HCR_channelList=='DAPI')[0][0]])
    io.imsave(path + "/2CY5.tif", images2[np.where(fto.HCR_channelList=='CY5')[0][0]])
    io.imsave(path + "/2CY3.tif", images2[np.where(fto.HCR_channelList=='CY3')[0][0]])
    io.imsave(path + "/2YFP.tif", images2[np.where(fto.HCR_channelList=='YFP')[0][0]])
    io.imsave(path + "/2DAPI.tif", images2[np.where(fto.HCR_channelList=='DAPI')[0][0]])

     |████████████████████████████████| 793 kB 6.9 MB/s 
     |████████████████████████████████| 381 kB 42.1 MB/s 
  Created wheel for ipdb: filename=ipdb-0.13.9-py3-none-any.whl size=11648 sha256=fd3c3ba8749073fb04766110430f5c73b17858f03a517e98318fa65a30efd3ed
  Stored in directory: /root/.cache/pip/wheels/65/cd/cc/aaf92acae337a28fdd2aa4d632196a59745c8c39f76eaeed01
Successfully built ipdb
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 

In [3]:
# 2) set sample and run functions
sampleBase = "516_D2_1_w1"
for block in range(7,8):
  generateAnnotationFiles(sampleBase, block)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: UserWarning: /content/drive/MyDrive/FateTrack_Main/test/0411/516_D2_1_w1/516_D2_1_w1_9trailing_block7of9/1CY5.tif is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: UserWarning: /content/drive/MyDrive/FateTrack_Main/test/0411/516_D2_1_w1/516_D2_1_w1_9trailing_block7of9/1CY3.tif is a low contrast image
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: /content/drive/MyDrive/FateTrack_Main/test/0411/516_D2_1_w1/516_D2_1_w1_9trailing_block7of9/2CY5.tif is a low contrast image


In [ ]:
sampleBase = "516_D2_1_w1"
block = 0
sample = sampleBase + "_9trailing_block" + str(block) + "of9"
fto = load_object('/content/drive/MyDrive/FateTrack_Main/test/0328/' + sample + '/' + sample + '.pkl')

KeyboardInterrupt: ignored

In [ ]:
fto.HCR_measurements.columns

Index(['Rd1_orig_label', 'Rd2_orig_label', 'Rd1_cytoplasm_label',
       'Rd2_cytoplasm_label', 'nuc_Rd1_centroid-0', 'nuc_Rd1_centroid-1',
       'nuc_Rd1_CY5_filled_area', 'nuc_Rd1_CY5_min_intensity',
       'nuc_Rd1_CY5_mean_intensity', 'nuc_Rd1_CY5_max_intensity',
       ...
       'cyto_Rd2_Brightfield_filled_area',
       'cyto_Rd2_Brightfield_min_intensity',
       'cyto_Rd2_Brightfield_mean_intensity',
       'cyto_Rd2_Brightfield_max_intensity',
       'cyto_Rd2_Brightfield_quantiles-0', 'cyto_Rd2_Brightfield_quantiles-1',
       'cyto_Rd2_Brightfield_quantiles-2', 'cyto_Rd2_Brightfield_quantiles-3',
       'cyto_Rd2_Brightfield_quantiles-4',
       'cyto_Rd2_Brightfield_sum_intensity'],
      dtype='object', length=252)

In [ ]:
generateAnnotationFiles("508_D5_1_w1_9trailing_block7of9")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: /content/drive/MyDrive/FateTrack_Training/annotations/508_D5_1_w1_9trailing_block7of9/508_D5_1_w1_9trailing_block7of9_2CY5.tif is a low contrast image


In [ ]:
io.imsave("/content/drive/MyDrive/FateTrack_Training/annotations/508_D5_1_w1_9trailing_block7of9/508_D5_1_w1_9trailing_block7of9_mask.tif", FatetrackObj.HCR_Rd1_DAPI_mask)

In [ ]:
# loading a FateTrack object just to poke around
FatetrackObj = load_object('/content/drive/MyDrive/FateTrack_Main/FateTrack_Training/output/508_D5_1_w1_9trailing_block7of9/508_D5_1_w1_9trailing_block7of9.pkl')


In [ ]:
FatetrackObj.HCR_Rd1_DAPI_mask

array([[   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ..., 6715, 6715, 6715],
       [   0,    0,    0, ..., 6715, 6715,    0],
       [   0,    0,    0, ..., 6715, 6715,    0]], dtype=uint16)

In [ ]:
io.imsave("/content/drive/MyDrive/new_folder/dapi.tif", FatetrackObj.HCR_Rd1_images[np.where(FatetrackObj.HCR_channelList=='DAPI')[0][0]])
io.imsave("/content/drive/MyDrive/new_folder/aldob.tif", FatetrackObj.HCR_Rd2_images[3])

In [ ]:
FatetrackObj.HCR_Rd1_images[np.where(FatetrackObj.HCR_channelList=='DAPI')[0][0]]


array([[0.21107178, 0.20422082, 0.20461297, ..., 0.16768236, 0.1721574 ,
        0.16459135],
       [0.20749637, 0.20507431, 0.2031136 , ..., 0.17111938, 0.16925093,
        0.16980454],
       [0.21245582, 0.20982616, 0.20375948, ..., 0.16788997, 0.16712875,
        0.16685194],
       ...,
       [0.13605722, 0.1367723 , 0.13492692, ..., 0.32225574, 0.28973097,
        0.25464575],
       [0.13647242, 0.1353652 , 0.13548053, ..., 0.29602832, 0.25978973,
        0.23070198],
       [0.13737205, 0.13568814, 0.13727978, ..., 0.27224603, 0.23556916,
        0.2157775 ]])

In [ ]:
FatetrackObj.HCR_Rd1_images

In [ ]:
FatetrackObj.HCR_Rd1_DAPI_mask

In [ ]:
FatetrackObj.HCR_channelList

array(['CY5', 'A594', 'CY3', 'YFP', 'DAPI', 'Brightfield'], dtype='<U11')

In [ ]:
FatetrackObj.HCR_Rd1_images[1]

array([[0.21690101, 0.22696353, 0.22663361, ..., 0.20337434, 0.19859051,
        0.19611612],
       [0.2254789 , 0.22795329, 0.23257215, ..., 0.20254954, 0.19496141,
        0.20650856],
       [0.22201475, 0.22943792, 0.22201475, ..., 0.20007515, 0.19793068,
        0.19479645],
       ...,
       [0.19991019, 0.19446653, 0.19710588, ..., 0.20947783, 0.20155978,
        0.20931287],
       [0.19347677, 0.18902287, 0.2018897 , ..., 0.2079932 , 0.20254954,
        0.20815816],
       [0.19892043, 0.2010649 , 0.19908539, ..., 0.22135491, 0.20238458,
        0.21277702]])